---
title: "Unindo Predição Estrutural, Docking Molecular e Análise de Interações: explorando o pacote _biolib_"
author: "Samuel Chagas"
date: "2024-07-15"
categories: [prediction, structural, python]
---

![](thumbnail.jpg)

In [27]:
#Installing py3Dmol using pip
!pip -q install py3Dmol
#Installing biopython using pip
!pip -q install biopython
#Installing pdb2pqr v3.0 using pip
!pip -q install pdb2pqr
#We will also install RDkit
!pip -q install rdkit-pypi



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not find a version that satisfies the requirement MolKit (from versions: none)
ERROR: No matching distribution found for MolKit


In [14]:
import py3Dmol

In [15]:
#Checking that pdb2pqr was properly installed
!pdb2pqr30 -h | awk 'NR==1{if($1=="usage:") print "PDB2PQR succesfully installed"; else if($1!="usage:") print "Something went wrong. Please install again"}'

PDB2PQR succesfully installed


In [17]:
#!pip install -q condacolab
#import condacolab
#condacolab.install_miniconda()

#Install MGLtools and OpenBabel from
#the bioconda repository
!conda install -c conda-forge -c bioconda openbabel mgltools zlib ncurses --yes

Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/samuel/anaconda3/envs/blogposts

  added / updated specs:
    - mgltools
    - ncurses
    - openbabel
    - zlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cairo-1.18.0               |       hebfffa5_3         961 KB  conda-forge
    expat-2.6.2                |       h59595ed_0         134 KB  conda-forge
    font-ttf-ubuntu-0.83       |       h77eed37_2         1.5 MB  conda-forge
    icu-75.1                   |       he02047a_0        11.6 MB  conda-forge
    libexpat-2.6.2             |       h59595ed_0          72 KB  conda-forge
    libglib-2.80.3             |       h8a4344b_1         3.7 MB  conda-forge
    libpng-1.6.43              |       h2797004_0         281 KB  conda-forge
    libxcb-1.16                |       hd590300_0

In [28]:
#Download and extract Autodock Vina from SCRIPPS
#Then, we set up an alias for vina to be treated as a native binary
#%%bash
!mkdir vina
!mkdir result
!wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64 --directory-prefix=vina
!chmod +x vina/vina_1.2.5_linux_x86_64
!wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_split_1.2.5_linux_x86_64 --directory-prefix=vina
!chmod +x vina/vina_split_1.2.5_linux_x86_64

mkdir: cannot create directory ‘vina’: File exists


In [29]:
%alias vina vina/vina_1.2.5_linux_x86_64
%alias vina_split vina/vina_1.2.5_linux_x86_64

In [33]:
import os
from Bio.PDB import *
pdbid = ['1hsg']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='result', file_format ="pdb", overwrite=True)
  os.rename("result/pdb"+s+".ent", "result/"+s+".pdb")

In [38]:
#This script will create a folder called "single-docking" for our experiment
#Then, it will print all "ATOM" and "TER" lines from a given PDB into a new file

#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
#if os.path.exists(singlepath):
#  print("path already exists")
#if not os.path.exists(singlepath):
#  os.mkdir(singlepath)
#  print("path was succesfully created")

#Now we assign a variable "protein" with the name and extension of our pdb
protein = "result/1hsg.pdb"

#And we use the following script to selectively print the lines that contain the
#string "ATOM" and "TER" into a new file inside our recently created folder
with open("result/1hsg_prot.pdb","w") as g:
  f = open(protein,'r')
  for line in f:
    row = line.split()
    if row[0] == "ATOM":
      g.write(line)
    elif row[0] == "TER":
      g.write("TER\n")
  g.write("END")
  print("file successfully created")

file successfully created


In [41]:
#First, using pdb2pqr to parameterize our receptor with AMBER99ff, maintaining
#the chain IDs and setting up the receptor at a pH of 7.4
!pdb2pqr30 --ff AMBER --keep-chain --titration-state-method propka --with-ph 7.4 result/1hsg_prot.pdb result/1hsg_prot.pqr

#Then, convert the .pqr file into a .pdbqt file while deleting non-polar
#hydrogens but without changing the AMBER parameters added to the protein
!prepare_receptor4.py -r result/1hsg_prot.pqr -o result/1hsg_prot.pdbqt -C -U nphs_lps -v

INFO:PDB2PQR v3.6.2: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: result/1hsg_prot.pdb
INFO:Setting up molecule.
INFO:Created biomolecule object with 198 residues and 1514 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 2 missing atoms in biomolecule.
INFO:Added atom OXT to residue PHE A 99 at coordinates 26.780, 34.489, -10.125
INFO:Added atom OXT to residue PHE B 99 at coordinates 26.380, 42.929, 4.902
INFO:Updating disulfide bridges.
INFO:Debumping biomolecule.
INFO:

In [44]:
#Downloading Indinavir from the DrugBank database in SMILES format
!wget -q https://www.drugbank.ca/structures/small_molecule_drugs/DB00224.smiles -P result/

#Print the SMILES of indinavir to see what it is all about
print(Path("result/DB00224.smiles").read_text())

CC(C)(C)NC(=O)[C@@H]1CN(CC2=CN=CC=C2)CCN1C[C@@H](O)C[C@@H](CC1=CC=CC=C1)C(=O)N[C@@H]1[C@H](O)CC2=CC=CC=C12


In [28]:
#Converting Indinavir from SMILES into a 3D PDB format
!obabel result/DB00224.smiles -opdbqt -O result/indinavir.pdbqt --gen3d best -p 7.4 --canonical
#Parameterizing and adding Gasteiger charges into our protein using MGLtools
#Adding -z leads to a rigid ligand without any torsions
#!python3 prepare_ligand4.py -l result/indinavir.mol2 -o result/indinavir.pdbqt -U nphs_lps -v
#NOTE: for some reason, MGLtools does not recognize the ligand when inside a different folder
#Here we are deleting the temporary PDB file required for generating the PDBQT file
#os.remove("indinavir.mol2")

1 molecule converted


In [69]:
!mglobabel -H

/home/samuel/anaconda3/envs/blogposts/bin/mglobabel: 36: exec: /home/samuel/anaconda3/envs/blogposts/bin/obabel: not found
